In [1]:
import requests
import pandas as pd

##### 1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [2]:
url = 'https://python.zgulde.net/api/v1/items'

In [3]:
max_pages = 3
target = 'items'

In [4]:
def create_payload_df(url, max_pages, target):
    page_list = []
    
    for i in range(1, max_pages + 1):
        response = requests.get(url + "?page=" + str(i))
        data = response.json()
        page_items = data['payload'][target]
        page_list += page_items
        
    return pd.DataFrame(page_list)

In [5]:
items = create_payload_df(url, max_pages, target)
items.shape

(50, 6)

In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  50 non-null     object 
 1   item_id     50 non-null     int64  
 2   item_name   50 non-null     object 
 3   item_price  50 non-null     float64
 4   item_upc12  50 non-null     object 
 5   item_upc14  50 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ KB


In [7]:
items.head(1)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


##### 2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [8]:
url = 'https://python.zgulde.net/api/v1/stores'
max_pages = 1
target = 'stores'

In [9]:
def create_payload_df(url, max_pages, target):
    page_list = []
    
    for i in range(1, max_pages + 1):
        response = requests.get(url + "?page=" + str(i))
        data = response.json()
        page_items = data['payload'][target]
        page_list += page_items
        
    return pd.DataFrame(page_list)

In [10]:
stores = create_payload_df(url, max_pages, target)
stores.shape

(10, 5)

In [11]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   store_address  10 non-null     object
 1   store_city     10 non-null     object
 2   store_id       10 non-null     int64 
 3   store_state    10 non-null     object
 4   store_zipcode  10 non-null     object
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [12]:
stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


##### 3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted

In [13]:
url = 'https://python.zgulde.net/api/v1/sales'

In [14]:
max_pages = 183
target = 'sales'

In [15]:
def create_payload_df(url, max_pages, target):
    page_list = []
    
    for i in range(1, max_pages + 1):
        response = requests.get(url + "?page=" + str(i))
        data = response.json()
        page_items = data['payload'][target]
        page_list += page_items
        
    return pd.DataFrame(page_list)

In [16]:
sales = create_payload_df(url, max_pages, target)
sales.shape

(913000, 5)

In [17]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   item         913000 non-null  int64  
 1   sale_amount  913000 non-null  float64
 2   sale_date    913000 non-null  object 
 3   sale_id      913000 non-null  int64  
 4   store        913000 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 34.8+ MB


In [18]:
sales.head(1)

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1


##### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [19]:
#moved to local drive due to git's size limit.
sales.to_csv(r'/Users/hector/Documents/cached_csvs/sales.csv')
items.to_csv(r'/Users/hector/Documents/cached_csvs/items.csv')
stores.to_csv(r'/Users/hector/Documents/cached_csvs/stores.csv')

##### 5. Combine the data from your three separate dataframes into one large dataframe.

In [20]:
#first merge: sales to stores
sales_to_stores = pd.merge(sales, stores, how='left', left_on='store', right_on='store_id')
sales_to_stores.shape

(913000, 10)

In [21]:
sales_to_stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 0 to 912999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           913000 non-null  int64  
 1   sale_amount    913000 non-null  float64
 2   sale_date      913000 non-null  object 
 3   sale_id        913000 non-null  int64  
 4   store          913000 non-null  int64  
 5   store_address  913000 non-null  object 
 6   store_city     913000 non-null  object 
 7   store_id       913000 non-null  int64  
 8   store_state    913000 non-null  object 
 9   store_zipcode  913000 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 76.6+ MB


In [22]:
#second merge
total_sales = pd.merge(sales_to_stores, items, how='left', left_on='item', right_on='item_id')
total_sales.shape

(913000, 16)

In [23]:
total_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 0 to 912999
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           913000 non-null  int64  
 1   sale_amount    913000 non-null  float64
 2   sale_date      913000 non-null  object 
 3   sale_id        913000 non-null  int64  
 4   store          913000 non-null  int64  
 5   store_address  913000 non-null  object 
 6   store_city     913000 non-null  object 
 7   store_id       913000 non-null  int64  
 8   store_state    913000 non-null  object 
 9   store_zipcode  913000 non-null  object 
 10  item_brand     913000 non-null  object 
 11  item_id        913000 non-null  int64  
 12  item_name      913000 non-null  object 
 13  item_price     913000 non-null  float64
 14  item_upc12     913000 non-null  object 
 15  item_upc14     913000 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 118.4+ MB


In [24]:
total_sales.to_csv(r'/Users/hector/Documents/cached_csvs/total_sales.csv')

##### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [25]:
opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
opsd.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


##### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.